In [1]:
#Import the required libraries
import pandas as pd 
from bs4 import BeautifulSoup
import requests

In [2]:
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [47]:
from sklearn.metrics.pairwise import cosine_similarity
import json

In [5]:
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\deepa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\deepa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\deepa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# 1. Collect and Preprocess Data

## Web Scraping tools used to gather dataset from online websites
### web_scrap is a function to scrap data from the website passed as parameter 
### Beautiful Soup is the Web Scraping tool used


In [16]:

def web_scrap(url1,class1,split_string):
    '''
    INPUT:
    url1: (String) - the URL of the website from which information is to be scraped
    class1: (String) - the class information in the div tag in html found in the URL specified
    split_string: (String) - the string is split using split_string
    
    OUTPUT:
    d1 - (DataFrame) A DataFrame containing the data scraped from the website 
    
    '''
    list_web_plus=['https://www.pluss.in/men','https://www.pluss.in/women/skirts','https://www.pluss.in/women/denim','https://www.pluss.in/women/kurtis','https://www.pluss.in/women/formal-trouser','https://www.pluss.in/women/tracks-pants','https://www.pluss.in/women/tunics','https://www.pluss.in/women/tops','https://www.pluss.in/women/t-shirts','https://www.pluss.in/men/bermuda','https://www.pluss.in/men/t-shirt','https://www.pluss.in/men/kurta-pyjamas','https://www.pluss.in/kids/boys-t-shirt']

    r=requests.get(url1)
    #print(r.text)
    soup=BeautifulSoup(r.content,'html.parser')
    title=soup.title.getText()
    #print(title)
    #cloth=soup.find('div',{'class':'comp tax-sc__recirc-list card-list mntl-document-card-list mntl-card-list mntl-block'})
    if(url1 in list_web_plus):
        cloth=soup.find_all('div',{'class':class1})
    else:
        cloth=soup.find('div',{'class':class1})
    #print(cloth)
    #d1['item_name']=str(title)
    list1=[]
    for a in cloth:
        if(split_string!=""):
            list1.append(a.getText().strip().split(split_string,2))
        else:
            list1.append(a.getText().strip())
    #print(list1)
    d1=pd.DataFrame(index=pd.Index(range(1,len(list1)+1)))

    #print(len(list1))
    #d1=pd.DataFrame(list1,columns=['index','Description'])
    d1=pd.DataFrame(list1)
    d1=d1.rename(columns={1:'Description'})  
    d1['link']=url1
    d1.insert(0,'item_name',title)
    
    #d1.drop(columns=[0],inplace=True)
    return d1

### The Websites from instore.co.in for the different clothing items are passed as a list to the function web_scrap specifying the class 
### value in the div tag found in the specified url and the split string used to split the items found in the website

In [17]:
def create_database():
    list_web_sites=["https://instore.co.in/collections/leggings","https://instore.co.in/collections/designer-dramas","https://instore.co.in/collections/classics","https://instore.co.in/collections/full-patiyalas"]
    d2=pd.DataFrame()
    for i in list_web_sites:
        dff=web_scrap(i,'grid grid--uniform grid--collection small--grid--flush','Quick view')
        d2=pd.concat([d2,dff],axis=0)
# The column 0 is removed from the DataFrame
    d2.drop(columns=[0],inplace=True)
    #d2
    d3=web_scrap("https://www.instyle.com/fashion/clothing/shirts-and-tops",'comp tax-sc__recirc-list card-list mntl-document-card-list mntl-card-list mntl-block','\n')
    d3=d3.rename(columns={0:'Description'})  
    #d3
    list_web_plus=['https://www.pluss.in/men','https://www.pluss.in/women/skirts','https://www.pluss.in/women/denim','https://www.pluss.in/women/kurtis','https://www.pluss.in/women/formal-trouser','https://www.pluss.in/women/tracks-pants','https://www.pluss.in/women/tunics','https://www.pluss.in/women/tops','https://www.pluss.in/women/t-shirts','https://www.pluss.in/men/bermuda','https://www.pluss.in/men/t-shirt','https://www.pluss.in/men/kurta-pyjamas','https://www.pluss.in/kids/boys-t-shirt']
    d4=pd.DataFrame()
    for j in list_web_plus:
        dff=web_scrap(j,"prodItem iprodItem","")
        d4=pd.concat([d4,dff],axis=0)
    d4=d4.rename(columns={0:'Description'})  
    #d4
    df=pd.concat([d2,d3,d4],axis=0)
    df['sno']=range(1,df.shape[0]+1)

    return df

## Preprocess the Text Data
### Data Cleaning

In [50]:
def data_analysis(df):
    #print("The shape of the DataFrame is ",df.shape)
    #print("The null values in the DataFrame are:")
    #print(df.isnull().sum())
    #print("There may be some null strings in the Description which has to be replaced as nan")
    df['Description']=df['Description'].replace("",np.nan)
    #print(df['Description'].isnull().sum())
    df=df.dropna()
    #print("The Shape of the DataFrame after null values are removed:",df.shape)

In [19]:
def tokenize(text):
    '''
    INPUT:
    text: (String) - the Text which is to be tokenized
    
    OUTPUT:
    clean_tokens - (String) Clean Tokens 
    
    Process:
    The text is converted to lower case and any special characters are removed. The sentence is then converted to tokens 
    and stopwords are removed. WordNetlemmatizer is applied to lemmatize and clean tokens are obtained.
    '''
    #col=df_content['doc_full_name']
    #print(col)
    clean_tokens=[]
    #for ln in text:
        #print(ln)
    sent=str(text)
    sent=sent.lower()
    sent1=re.sub('[0-9]','',sent)
    sp=re.compile('<.*?(#-,)>')
    sent2=re.sub(sp,'',sent1)
    tokens=word_tokenize(sent2)
    words=[w for w in tokens if len(w)>3 if w not in stopwords.words('english')]
    lemmatizer=WordNetLemmatizer()
    for tok in words:
        cl=lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(cl)
    clean_tokens=' '.join(clean_tokens)  # s=set(clean_tokens)
    return clean_tokens

# 2. Measure Similarity and retrieve Ranked Results

In [54]:
def similarity_rank(df,inp_text):
    '''
    INPUT:
    df: (DataFrame) - the DataFrame containing the scraped data from websites
    inp_text: (String) -  The input string for which similar url's are to be determined
    
    OUTPUT:
    suggested_urls - json - A json object of suggested URL's that are similar to the input text
    
    
    '''
    input_cl=tokenize(inp_text)
    #print(input_cl)
    cv=CountVectorizer()
    word_count=cv.fit_transform(df['Description'])
    #print(word_count.shape)
    cv.fit(df['Description'])
    inp_vect=cv.transform([input_cl])
    ds_vect=cv.transform(df['Description'])
    sim1=cosine_similarity(inp_vect,ds_vect)
    sort_ind=sim1.argsort()[0][::-1]
    #print(sort_ind)
    sugg=[]
    for i in sort_ind[:15]:
        u=df[df['sno']==i].link.values[0]
        if u not in sugg:
            sugg.append(u)
    suggested={
        'suggestions':sugg
    }
    suggested_urls=json.dumps(suggested)
    #sg=sorted(set(map(tuple,sugg)),reverse=True)
    return suggested_urls

In [55]:
def Clothing_similarity_search(input_text):
    list_web_plus=['https://www.pluss.in/men','https://www.pluss.in/women/skirts','https://www.pluss.in/women/denim','https://www.pluss.in/women/kurtis','https://www.pluss.in/women/formal-trouser','https://www.pluss.in/women/tracks-pants','https://www.pluss.in/women/tunics','https://www.pluss.in/women/tops','https://www.pluss.in/women/t-shirts','https://www.pluss.in/men/bermuda','https://www.pluss.in/men/t-shirt','https://www.pluss.in/men/kurta-pyjamas','https://www.pluss.in/kids/boys-t-shirt']
    
    df_final=create_database()
    data_analysis(df_final)
    df_final['Description']=df_final['Description'].apply(tokenize)
    urls=similarity_rank(df_final,input_text)
    print(urls)

In [56]:
Clothing_similarity_search("This is a stylish and comfortable t-shirt in black")

{"suggestions": ["https://www.pluss.in/kids/boys-t-shirt", "https://www.pluss.in/men/kurta-pyjamas", "https://www.pluss.in/women/tops", "https://www.pluss.in/men/t-shirt", "https://www.pluss.in/men", "https://www.pluss.in/women/t-shirts"]}
